In [40]:
import transformers # HuggingFace!
from transformers import AutoTokenizer # a librabry that has tokenizers for all of the different models
from transformers import pipeline
from transformers import AutoModelForSequenceClassification
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
#import pdb

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/LizzieBlaschke/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

path = "clean_IMDBdataset.csv"
df = pd.read_csv(path, sep=",")

In [19]:
#Read a comma-separated values (csv) file into DataFrame
path = "clean_IMDBdataset.csv"
df = pd.read_csv(path, sep=",")

In [20]:
#This changes all of the Y labels to be 0s or 1s
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

In [34]:

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(df, df, test_size=0.2, random_state=30)

X_test,X_valid,Y_test, Y_valid = train_test_split(X_test, Y_test, test_size=0.5, random_state=30)

X_tr_dataset = Dataset.from_pandas(X_train)
X_test_dataset = Dataset.from_pandas(X_test)
X_v_dataset = Dataset.from_pandas(X_valid)

def tokenize_function(examples):
    return tokenizer(examples["Phrase"], padding="max_length", truncation=True)


# Tokenizes the data
tokenizedX_train = X_tr_dataset.map(tokenize_function, batched=True)
tokenizedX_test = X_test_dataset.map(tokenize_function, batched=True)
tokenizedX_valid = X_v_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [38]:
# Remove the column with the actual reviews (as opposed to the review ids) because the model does not accept raw text as an input!
tokenizedX_trainE = tokenizedX_train.remove_columns(["Phrase"])
tokenizedX_testE = tokenizedX_test.remove_columns(["Phrase"])
tokenizedX_validE = tokenizedX_valid.remove_columns(["Phrase"])

# Removes another unnecesary column
tokenizedX_trainE = tokenizedX_trainE.remove_columns(["__index_level_0__"])
tokenizedX_testE = tokenizedX_testE.remove_columns(["__index_level_0__"])
tokenizedX_validE = tokenizedX_validE.remove_columns(["__index_level_0__"])


# Makes sure the columns are titled in the way that the model expects
tokenizedX_trainf = tokenizedX_trainE.rename_column("Sentiment", "labels")
tokenizedX_testf = tokenizedX_testE.rename_column("Sentiment", "labels")
tokenizedX_validf = tokenizedX_validE.rename_column("Sentiment", "labels")



# Sets the format of the dataset to return PyTorch tensors instead of lists
tokenizedX_trainf.set_format("torch")
tokenizedX_testf.set_format("torch")
tokenizedX_validf.set_format("torch")

<class 'datasets.arrow_dataset.Dataset'>


In [39]:
# You might need to create a smaller subset of the dataset if it's going really slow

train_dataloader = DataLoader(tokenizedX_trainf, shuffle=True, batch_size=8)
# I don't want the eval DataLoader to be my validation, right?
valid_dataloader = DataLoader(tokenizedX_validf, batch_size=8)


In [41]:
# Loading the model with the expected number of labels, for the IMDb dataset, there are two different labels, 0 or 1!
# Maybe I could try loading bert-base-cased too?

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [43]:
# Creating the optimizer
from torch.optim import AdamW

optim = AdamW(model.parameters(), lr = 5e-5)

In [44]:
from transformers import get_scheduler

# Can adjust this is it's not quite right...
num_epochs = 3
num_train_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name = "linear", optimizer = optim, num_warmup_steps = 0, num_training_steps = num_train_steps
)


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)


In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_train_steps))

model.train()
for ep in range(num_epochs):
  for batch in train_dataloader:
    #print(batch.keys())
    #for k in batch:
      #print(k, batch[k]) 
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward

    optim.step()
    lr_scheduler.step()
    optim.zero_grad()
    progress_bar.update(1)

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()